In [2]:
import pandas as pd
import numpy as np

# XGBRegressor - NaN en zéros - 2023/11/19


- Import des données


In [2]:
path = "../../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate
x_predict = pd.read_csv(
    path + "/Xtest.csv", sep=","
)  # features pour la prédiction

- Mise en forme de x_data


In [3]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data = x_data.drop("way", axis=1) #  on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les Nan de hour par des 0 et les heures par des entiers

#on remplace les Nan des pxqx par 0
x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]].fillna(0)

# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Mise en forme de x_predict


In [4]:
x_predict = x_predict.drop("date", axis=1)  # on supprime la colonne date
x_predict = x_predict.drop("way", axis=1) #  on supprime la colonne way
x_predict["hour"] = x_predict["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les Nan de hour par des 0 et les heures par des entiers

#on remplace les Nan des pxqx par 0
x_predict[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_predict[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]].fillna(0)

# on tranforme les identifiants de gare en entiers
x_predict["station"] = x_predict["station"].astype("category")
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)

- Apprentissage du modèle retenu

In [5]:
from xgboost import XGBRegressor

In [6]:
XGBR3 = XGBRegressor( 
        n_estimators = 4000,learning_rate = 0.02,
        max_depth=6, min_child_weight=1, 
        gamma = 0.0001, colsample_bytree=1, subsample=0.8714285714285714,
        reg_alpha = 0.08071428571428571, reg_lambda= 0.15142857142857144
    )
XGBR3.fit(x_data, y_data)
XGBR3.score(x_data, y_data)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtyp

0.9964187419492565

- Prédiction et export


In [7]:
y_predict = XGBR3.predict(x_predict.values)

In [9]:
from export import export_to_csv

export_to_csv(y_predict, "xgbregressor_nan_zeros_23_11_20")

# XGBRegressor - NaN en mean - 2023/11/21

In [18]:
path = "../../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate
x_predict = pd.read_csv(
    path + "/Xtest.csv", sep=","
)  # features pour la prédiction

- Mise en forme de x_data

In [19]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data = x_data.drop("way", axis=1) #  on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(
    lambda x: np.nan if type(x) == float else int(x[:2])
) # on remplace les heures par des entiers en conservant les NaNs

# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Remplacement des NaNs de x_data

In [20]:
from sklearn.impute import SimpleImputer
SIM = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
SIMF = SimpleImputer(missing_values=np.nan, strategy='most_frequent', add_indicator=True) # pour les heures

In [21]:
x_data_full = SIM.fit_transform(x_data[['train', 'station', 'composition', 'p1q0', 'p2q0', 'p3q0', 'p0q1', 'p0q2', 'p0q3']])
x_data_full = np.append(x_data_full, SIMF.fit_transform(x_data['hour'].to_numpy().reshape(-1, 1)), axis = 1)

- Mise en forme de x_predict

In [22]:
x_predict = x_predict.drop("date", axis=1)  # on supprime la colonne date
x_predict = x_predict.drop("way", axis=1) #  on supprime la colonne way
x_predict["hour"] = x_predict["hour"].apply(
    lambda x: np.nan if type(x) == float else int(x[:2])
) # on remplace les heures par des entiers en conservant les NaNs

# on tranforme les identifiants de gare en entiers
x_predict["station"] = x_predict["station"].astype("category")
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)

- Remplacenent des NaNs de x_predict

In [23]:
x_predict_full = SIM.fit_transform(x_predict[['train', 'station', 'composition', 'p1q0', 'p2q0', 'p3q0', 'p0q1', 'p0q2', 'p0q3']])
x_predict_full = np.append(x_predict_full, SIMF.fit_transform(x_predict['hour'].to_numpy().reshape(-1, 1)), axis = 1)

- Apprentissage du modèle retenu

In [24]:
from xgboost import XGBRegressor

In [25]:
XGBR = XGBRegressor( 
        learning_rate= 0.024999999999999998, n_estimators= 2250,
        max_depth=8, min_child_weight=5, gamma = 0.0001, colsample_bytree=1, subsample=0.8666666666666667,
        reg_alpha= 0.13333333333333333, reg_lambda= 0.8222222222222222
    )
XGBR.fit(x_data_full, y_data)
XGBR.score(x_data_full, y_data)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


0.9964618670756457

- Prédiction et export

In [26]:
y_predict_full = XGBR.predict(x_predict_full)

In [27]:
from export import export_to_csv

export_to_csv(y_predict_full, "xgbregressor_nan_mean_23_11_21")

# XGBRegressor - NaN avec KNN - 2023/11/22

In [3]:
path = "../../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate
x_predict = pd.read_csv(
    path + "/Xtest.csv", sep=","
)  # features pour la prédiction

- Mise en forme de x_data

In [4]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data = x_data.drop("way", axis=1) #  on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les heures par des entiers et les Nans en 0

# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Remplacement des NaNs de x_data

In [5]:
from sklearn.impute import KNNImputer
KNNI = KNNImputer(missing_values=np.nan, add_indicator=True)
x_data_full = KNNI.fit_transform(x_data)

- Mise en forme de x_predict

In [6]:
x_predict = x_predict.drop("date", axis=1)  # on supprime la colonne date
x_predict = x_predict.drop("way", axis=1) #  on supprime la colonne way
x_predict["hour"] = x_predict["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les heures par des entiers et les Nans par zéro

# on tranforme les identifiants de gare en entiers
x_predict["station"] = x_predict["station"].astype("category")
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)

- Remplacenent des NaNs de x_predict

In [7]:
x_predict_full = KNNI.fit_transform(x_predict)

- Apprentissage du modèle retenu

In [8]:
from xgboost import XGBRegressor

In [18]:
XGBR = XGBRegressor( 
        learning_rate= 0.02, n_estimators= 3760,
        max_depth=6, min_child_weight=4.333333333333333, gamma= 8.53061224489796e-05,
        colsample_bytree=1, subsample=1,
        reg_alpha= 0.15263157894736842, reg_lambda= 0.08105263157894738
    )
XGBR.fit(x_data_full, y_data)
XGBR.score(x_data_full, y_data)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


0.9949103657693115

- Prédiction et export

In [19]:
y_predict_full = XGBR.predict(x_predict_full)

In [20]:
from export import export_to_csv

export_to_csv(y_predict_full, "xgbregressor_nan_knn_23_11_22")

- Par curiosité, on exporte aussi une prédiction sur un xgbregressor avec tous ses paramètres par défaut

In [9]:
from export import export_to_csv
XGBR = XGBRegressor()
XGBR.fit(x_data_full, y_data)
print(XGBR.score(x_data_full, y_data))
y_predict_full = XGBR.predict(x_predict_full)

export_to_csv(y_predict_full, "xgbregressor_nan_knn_v2_23_11_22")

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


0.9926941455341106
